# Seasonal Decompose Feature Extractor

* Seasonal Strength, Trend Strength, Residual Variability

* Source Link via Rob Hyndman: 
    * https://otexts.com/fpp2/seasonal-strength.html

In [1]:
import polars as pl 

from polars_ts.decomposition.seasonal_decompose_features import seasonal_decompose_features

In [2]:
df = pl.read_parquet("https://datasets-nixtla.s3.amazonaws.com/m4-hourly.parquet")

# Plot Results

* Now we see a relative measure of Seasonality/Trend Over All Series.

In [3]:
simple_decompose_features = seasonal_decompose_features(df = df,
id_col='unique_id',
time_col='ds',
target_col='y',
ts_freq=24,
mode='simple')

mstl_decompose_features = seasonal_decompose_features(df = df,
id_col = 'unique_id',
time_col = 'ds',
target_col = 'y',
ts_freq = 24,
mode='mstl',
seasonal_freqs = [24,48,167,24*28])

ColumnNotFoundError: seasonal

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'select' <---
DF ["unique_id", "ds", "y", "trend"]; PROJECT */6 COLUMNS

# Plot Naive Decomposition Features

* 24 period seasonality 
* Simple Moving Average Trend (Naive)

In [4]:
import hvplot.polars 

seas = simple_decompose_features['seasonal_strength'].hvplot.hist(alpha=.6,bins=50,title="Hyndman's Seasonal and Trend Strength")

trend = simple_decompose_features['trend_strength'].hvplot.hist(alpha=.6, bins = 50)

seas * trend 

:Overlay
   .Histogram.I  :Histogram   [seasonal_strength]   (Count)
   .Histogram.II :Histogram   [trend_strength]   (Count)

# Plot MSTL Features 
    * Multiple Seasonal Components can be examined 
    * LOESS Trend as Opposed to MA Trend

In [5]:
# Assuming `mstl_feats` is a DataFrame containing the columns
# 'seasonal24_strength', 'seasonal48_strength', 'seasonal167_strength'

# Customizing the plot
hist_plot = mstl_decompose_features.hvplot.hist(
    y=['seasonal24_strength','seasonal48_strength', 'seasonal167_strength','seasonal672_strength'],
    title='Normalized MSTL Components of All Series',
    color=['#1f77b4', '#ff7f0e', '#2ca02c'],  # Custom color palette
    alpha=0.6,  # Transparency for the bars
    legend='top_left',  # Place legend in the top-left corner
    width=800,  # Set a custom width for the plot
    height=500,  # Set a custom height for the plot
    bins=50,  # Number of bins
    xlabel='Seasonal Strength S(t)',  # Label for the x-axis
    ylabel='Frequency',  # Label for the y-axis
    grid=True,  # Enable gridlines
    invert_axes=False  # Standard axes (not inverted)
)

#plot trend strength 
trend_plot = mstl_decompose_features.hvplot.hist(
    y=['trend_strength'],
    title='Normalized LOESS Component Of All Series',
    color=['#1f77b4', '#ff7f0e', '#2ca02c'],  # Custom color palette
    alpha=0.6,  # Transparency for the bars
    legend='top_left',  # Place legend in the top-left corner
    width=800,  # Set a custom width for the plot
    height=500,  # Set a custom height for the plot
    bins=50,  # Number of bins
    xlabel='Trend Strength T(t)',  # Label for the x-axis
    ylabel='Frequency',  # Label for the y-axis
    grid=True,  # Enable gridlines
    invert_axes=False  # Standard axes (not inverted)
)

# Display the plot
hist_plot + trend_plot

:Layout
   .NdOverlay.I  :NdOverlay   [Variable]
      :Histogram   [seasonal672_strength]   (Count)
   .NdOverlay.II :NdOverlay   [Variable]
      :Histogram   [trend_strength]   (Count)